In [ ]:
import numpy as np
from skimage import io, util, color
import matplotlib.pyplot as plt

# ==============================================================================
# 1. Implementação das Operações de Morfologia Matemática (Numpy Puro)
# ==============================================================================

def pad_image(img, pad_width=1):
    """Adiciona bordas de zeros à imagem para processamento de vizinhança."""
    return np.pad(img, pad_width, mode='constant', constant_values=0)

def logical_erosion(img, kernel):
    """
    Erosão Binária: Apenas pixels onde o kernel 'cabe' completamente permanecem 1.
    Equivalente a verificar se todos os vizinhos correspondentes aos 1s do kernel são 1.
    """
    h, w = img.shape
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    
    padded_img = pad_image(img, pad_width=1) # Assumindo kernel 3x3 para simplificar
    result = np.zeros_like(img)
    
    # Vamos usar shifts (deslocamentos) para evitar loops lentos pixel a pixel
    # Um pixel (x,y) sobrevive se, para todo (i,j) onde kernel[i,j]==1, img[x+i, y+j]==1
    
    # Criamos uma 'stack' de visões deslocadas da imagem
    shifts = []
    for i in range(kh):
        for j in range(kw):
            if kernel[i, j] == 1:
                # Extrai a fatia correspondente ao deslocamento
                # O shift é relativo ao centro.
                # Ex: i=0 (topo), desloca a imagem "para cima" na visão do array
                shifts.append(padded_img[i:i+h, j:j+w])
    
    if not shifts:
        return img # Kernel vazio
        
    # A erosão é o AND lógico de todos os deslocamentos onde o kernel é 1
    result = np.logical_and.reduce(shifts).astype(np.uint8)
    return result

def logical_dilation(img, kernel):
    """
    Dilatação Binária: Se qualquer pixel sob o kernel for 1, o centro vira 1.
    Equivalente a inverter o kernel e verificar sobreposição, ou OR lógico.
    """
    h, w = img.shape
    kh, kw = kernel.shape
    padded_img = pad_image(img, pad_width=1)
    
    shifts = []
    for i in range(kh):
        for j in range(kw):
            if kernel[i, j] == 1:
                shifts.append(padded_img[i:i+h, j:j+w])
    
    if not shifts:
        return img
        
    # A dilatação é o OR lógico
    result = np.logical_or.reduce(shifts).astype(np.uint8)
    return result

def logical_opening(img, kernel):
    """Abertura: Erosão seguida de Dilatação."""
    return logical_dilation(logical_erosion(img, kernel), kernel)

def logical_closing(img, kernel):
    """Fechamento: Dilatação seguida de Erosão."""
    return logical_erosion(logical_dilation(img, kernel), kernel)

def hit_or_miss(img, kernel):
    """
    Transformada Hit-or-Miss.
    Kernel deve ter: 1 (foreground), -1 (background), 0 (don't care).
    Resultado = (Erosão(Img, K_fg)) AND (Erosão(~Img, K_bg))
    """
    # Máscara do que deve ser 1 (foreground)
    k_fg = (kernel == 1).astype(np.uint8)
    
    # Máscara do que deve ser 0 (background). 
    # Nota: No input user, background é -1.
    k_bg = (kernel == -1).astype(np.uint8)
    
    # Erosão da imagem original com a parte FG do kernel
    e1 = logical_erosion(img, k_fg)
    
    # Erosão do complemento da imagem com a parte BG do kernel
    # Inverter imagem: 1 vira 0, 0 vira 1
    img_inv = 1 - img
    e2 = logical_erosion(img_inv, k_bg)
    
    return np.logical_and(e1, e2).astype(np.uint8)

# ==============================================================================
# 2. Algoritmos de Afinamento (Thinning) e Poda (Pruning)
# ==============================================================================

def get_thinning_kernels():
    """
    Retorna a sequência de 8 kernels rotacionados para afinamento.
    Baseado no algoritmo padrão de afinamento morfológico (Golay ou similar).
    1: deve ser pixel branco
    -1: deve ser pixel preto
    0: don't care
    """
    # Kernel base (L1)
    k1 = np.array([
        [-1, -1, -1],
        [ 0,  1,  0],
        [ 1,  1,  1]
    ])
    
    kernels = [k1]
    # Gera as 8 rotações (45 graus não é direto em array, usamos rotação 90 + intermediários)
    # Lista explicita para garantir a ordem correta de "peeling"
    
    k2 = np.array([[-1, -1, 0], [-1, 1, 1], [0, 1, 1]])
    k3 = np.rot90(k1)
    k4 = np.rot90(k2)
    k5 = np.rot90(k3)
    k6 = np.rot90(k4)
    k7 = np.rot90(k5)
    k8 = np.rot90(k6)

    return [k1, k2, k3, k4, k5, k6, k7, k8]

def thinning(img):
    """
    Realiza o afinamento iterativo até convergência (esqueletização).
    Imagem - HitOrMiss(Kernel_i)
    """
    skeleton = img.copy()
    kernels = get_thinning_kernels()
    
    while True:
        prev_skeleton = skeleton.copy()
        for k in kernels:
            hm = hit_or_miss(skeleton, k)
            # Subtrai os pixels encontrados pelo Hit-or-Miss (apaga bordas)
            # skeleton = skeleton AND (NOT hm)
            skeleton = np.logical_and(skeleton, np.logical_not(hm)).astype(np.uint8)
        
        # Se não houve mudança após passar todos os kernels, paramos
        if np.array_equal(skeleton, prev_skeleton):
            break
            
    return skeleton

def get_endpoint_kernels():
    """
    Retorna kernels para detectar 'pontas' (endpoints) de linhas.
    Uma ponta é um pixel 1 conectado a apenas um outro pixel 1 (conectividade-8).
    """
    # Estrutura: Pixel central 1, um vizinho 1, resto 0 (-1 aqui).
    k1 = np.array([
        [-1, -1, -1],
        [-1,  1, -1],
        [-1,  1, -1]
    ]) # Ponta apontando pra cima (conectado em baixo)
    
    k2 = np.array([[-1, -1, -1], [-1, 1, -1], [1, -1, -1]]) # Diagonal
    
    # Gerar todas as 8 rotações
    kernels = []
    curr_k1 = k1
    curr_k2 = k2
    for _ in range(4):
        kernels.append(curr_k1)
        kernels.append(curr_k2)
        curr_k1 = np.rot90(curr_k1)
        curr_k2 = np.rot90(curr_k2)
        
    return kernels

def pruning(img, iterations=5):
    """
    Realiza a poda (remoção de spurs/ramificações curtas).
    1. Detecta endpoints.
    2. Remove endpoints.
    3. Repete N vezes.
    """
    pruned = img.copy()
    ep_kernels = get_endpoint_kernels()
    
    for _ in range(iterations):
        endpoints_map = np.zeros_like(pruned)
        
        # Encontra todos os endpoints atuais
        for k in ep_kernels:
            hm = hit_or_miss(pruned, k)
            endpoints_map = np.logical_or(endpoints_map, hm)
        
        # Remove os endpoints detectados
        pruned = np.logical_and(pruned, np.logical_not(endpoints_map)).astype(np.uint8)
        
    return pruned

# ==============================================================================
# 3. Bloco Principal de Teste e IO
# ==============================================================================

def main():
    # --- IO: Carregar Imagem ---
    # Substitua 'digital.png' pelo caminho da sua imagem
    try:
        # Tenta carregar imagem local ou gera uma dummy se não existir para o exemplo rodar
        path = 'digital.png' 
        image_raw = io.imread(path)
        
        # Converter para cinza se for RGB e binarizar
        if len(image_raw.shape) == 3:
            image_gray = color.rgb2gray(image_raw)
        else:
            image_gray = image_raw
            
        # Binarização simples (limiar global)
        thresh = image_gray.mean() # ou use um valor fixo ex: 0.5
        image_bin = (image_gray > thresh).astype(np.uint8)
        
        # Garante que o fundo é 0 e a digital (traço) é 1
        # Se a imagem for fundo branco e traço preto, inverta:
        if image_bin[0,0] == 1: 
            image_bin = 1 - image_bin

    except FileNotFoundError:
        print("Imagem 'digital.png' não encontrada. Gerando dados sintéticos...")
        image_bin = np.zeros((200, 200), dtype=np.uint8)
        # Desenha uma cruz grossa e um quadrado para simular traços
        image_bin[40:160, 90:110] = 1
        image_bin[90:110, 40:160] = 1
        image_bin[50:80, 50:80] = 1 

    # --- Divisão da Imagem (Dividir em 4, pegar 1) ---
    h, w = image_bin.shape
    h_mid, w_mid = h // 2, w // 2
    
    # Pegando o quadrante superior esquerdo para processar
    crop = image_bin[0:h_mid, 0:w_mid]
    
    print(f"Processando quadrante de tamanho: {crop.shape}")

    # --- Pipeline: Afinamento -> Poda ---
    
    # 1. Pré-processamento (Opcional, mas comum: Fechamento para unir falhas no traço)
    struct_elem = np.ones((3,3), dtype=np.uint8)
    img_closed = logical_closing(crop, struct_elem)
    
    # 2. Afinamento (Skeletonization)
    print("Executando Afinamento...")
    skeleton = thinning(img_closed)
    
    # 3. Poda (Pruning)
    print("Executando Poda...")
    # Remove pequenos ramos de até 3 pixels
    pruned_skeleton = pruning(skeleton, iterations=3) 

    # --- Visualização ---
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    ax = axes.ravel()

    ax[0].imshow(crop, cmap='gray')
    ax[0].set_title('Original (Recorte)')
    
    ax[1].imshow(skeleton, cmap='gray')
    ax[1].set_title('Afinamento (Skeleton)')
    
    ax[2].imshow(pruned_skeleton, cmap='gray')
    ax[2].set_title('Poda (Resultado Final)')

    for a in ax: a.axis('off')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


: 

In [ ]:
import numpy as np
from skimage import io, color, util
import matplotlib.pyplot as plt

# ==============================================================================
# 1. Implementação de Morfologia em Nível de Cinza (Numpy Puro)
# ==============================================================================

def pad_image(img, pad_h, pad_w, mode='edge'):
    """
    Adiciona bordas à imagem. 
    Para Grayscale, 'edge' (repetir borda) ou 'reflect' é melhor que 'constant' (0),
    pois 0 afetaria erroneamente o cálculo do Mínimo (Erosão).
    """
    return np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode=mode)

def get_shifted_views(img, kernel_shape):
    """
    Gera uma lista de 'visões' deslocadas da imagem correspondentes 
    a cada posição do kernel. Técnica vetorizada para evitar loops por pixel.
    """
    h, w = img.shape
    kh, kw = kernel_shape
    ph, pw = kh // 2, kw // 2
    
    padded = pad_image(img, ph, pw, mode='edge')
    
    shifts = []
    # Para cada posição no kernel (que assumimos ser plano/flat com valor 1)
    for i in range(kh):
        for j in range(kw):
            # Extrai a fatia deslocada
            # Se kernel[i, j] for usado (estruturante não retangular), checar aqui.
            # Assumiremos elemento estruturante retangular cheio (todos True).
            shifts.append(padded[i:i+h, j:j+w])
            
    # Retorna uma pilha (stack) de imagens deslocadas: (K*K, H, W)
    return np.stack(shifts, axis=0)

def gray_erosion(img, kernel_size=(3,3)):
    """
    Erosão em Nível de Cinza (Flat Structuring Element).
    Definição: Mínimo valor na vizinhança definida pelo kernel.
    """
    # 1. Obter todas as sobreposições da vizinhança
    stack = get_shifted_views(img, kernel_size)
    
    # 2. Calcular o MÍNIMO ao longo do eixo da pilha
    return np.min(stack, axis=0)

def gray_dilation(img, kernel_size=(3,3)):
    """
    Dilatação em Nível de Cinza (Flat Structuring Element).
    Definição: Máximo valor na vizinhança definida pelo kernel.
    """
    stack = get_shifted_views(img, kernel_size)
    
    # Calcular o MÁXIMO ao longo do eixo da pilha
    return np.max(stack, axis=0)

def gray_opening(img, kernel_size):
    """
    Abertura em Nível de Cinza: Erosão seguida de Dilatação.
    Remove picos de brilho menores que o elemento estruturante.
    """
    eroded = gray_erosion(img, kernel_size)
    return gray_dilation(eroded, kernel_size)

def gray_closing(img, kernel_size):
    """
    Fechamento em Nível de Cinza: Dilatação seguida de Erosão.
    Preenche buracos escuros menores que o elemento estruturante.
    """
    dilated = gray_dilation(img, kernel_size)
    return gray_erosion(dilated, kernel_size)

def top_hat(img, kernel_size):
    """
    White Top-Hat Transform.
    Fórmula: Imagem Original - Abertura(Imagem).
    Objetivo: Destacar objetos claros (menores que o kernel) em fundo escuro 
    ou corrigir iluminação não uniforme.
    """
    opening = gray_opening(img, kernel_size)
    
    # Converter para int16 para evitar underflow (negativos) na subtração, depois clipar
    res = img.astype(np.int16) - opening.astype(np.int16)
    res = np.clip(res, 0, 255).astype(np.uint8)
    
    return res

# ==============================================================================
# 2. Execução e Teste
# ==============================================================================

def main():
    # --- IO: Carregar Imagem ---
    img_path = 'carro.jpg' # Nome genérico, substitua pelo seu arquivo
    
    try:
        image_raw = io.imread(img_path)
    except FileNotFoundError:
        print(f"Erro: Imagem '{img_path}' não encontrada.")
        print("Criando uma imagem de teste sintética (Simulando placa iluminada)...")
        # Cria gradiente (fundo ruim)
        x = np.linspace(0, 1, 300)
        y = np.linspace(0, 1, 300)
        X, Y = np.meshgrid(x, y)
        bg = (0.2 * X + 0.3 * Y) * 255
        image_raw = bg.astype(np.uint8)
        
        # Adiciona objetos claros (letras da placa)
        image_raw[100:150, 100:120] = 200 # Objeto 1
        image_raw[100:150, 130:150] = 220 # Objeto 2
        
        # Adiciona ruído
        noise = np.random.randint(0, 20, (300, 300))
        image_raw = np.clip(image_raw + noise, 0, 255).astype(np.uint8)

    # Converter para escala de cinza se necessário
    if len(image_raw.shape) == 3:
        image_gray = (color.rgb2gray(image_raw) * 255).astype(np.uint8)
    else:
        image_gray = image_raw

    # --- Divisão da Imagem (1/4) ---
    h, w = image_gray.shape
    h_mid, w_mid = h // 2, w // 2
    
    # Selecionando o quadrante superior esquerdo para teste
    crop = image_gray[0:h_mid, 0:w_mid]
    print(f"Processando quadrante de dimensão: {crop.shape}")

    # --- Aplicação do Top-Hat ---
    # O tamanho do kernel define o tamanho dos objetos que queremos ISOLAR.
    # Objetos maiores que o kernel serão considerados "fundo" e removidos.
    # Objetos menores que o kernel serão realçados.
    k_size = (15, 15) # Ajuste conforme o tamanho das letras da placa/detalhes
    
    print("Calculando Abertura...")
    # Passo intermediário apenas para visualização
    img_opening = gray_opening(crop, k_size) 
    
    print("Calculando Top-Hat...")
    img_tophat = top_hat(crop, k_size)

    # --- Visualização ---
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    ax = axes.ravel()

    ax[0].imshow(crop, cmap='gray')
    ax[0].set_title('Original (1/4 da Imagem)')

    ax[1].imshow(img_opening, cmap='gray')
    ax[1].set_title(f'Abertura (Background estimado)\nKernel {k_size}')

    ax[2].imshow(img_tophat, cmap='gray')
    ax[2].set_title('Top-Hat (Detalhes realçados)')

    for a in ax: a.axis('off')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()
